In [8]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# pymc
import pymc as pm
import pytensor
import pytensor.tensor as pt
#import pymc.sampling_jax
# import numpyro
# import blackjax
# import jax
# import arviz as az

# sklearn
from sklearn import preprocessing
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import Lasso

#statsmodels
import statsmodels.graphics.tsaplots as smg
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.stattools import kpss
import statsmodels.api as sm
import statsmodels.formula.api as smf



# plots
settings = {'figure.figsize':(14,4),
            'figure.dpi':144,
            'figure.facecolor':'w',
            'axes.spines.top':False,
            'axes.spines.bottom':False,
            'axes.spines.left':False,
            'axes.spines.right':False,
            'axes.grid':True,
            'grid.linestyle':'--',
            'grid.linewidth':0.5,
            'figure.constrained_layout.use':True}
plt.rcParams.update(settings)

In [9]:
def readdata():
    data = pd.read_csv(r'../data/hugo_data.csv', index_col=[0], date_format="mm/dd/yyyy")
    return data

raw_data = readdata()
print(raw_data)
print(raw_data.info())

           Start-ups         GDP         U     NAIRU  Job_vacancy        HICP  \
1/1/2006         NaN  179.989000  6.433333  4.869898          NaN   86.326667   
4/1/2006         NaN  182.917000  6.133333  4.877819          NaN   87.216667   
7/1/2006         NaN  183.853000  5.933333  4.885739          NaN   87.123333   
10/1/2006        NaN  185.449000  5.866667  4.893660          NaN   87.276667   
1/1/2007     46865.0  187.544000  5.633333  4.936292          NaN   87.610000   
...              ...         ...       ...       ...          ...         ...   
7/1/2023     64570.0  233.457000  3.633333  3.957463          4.8  129.320000   
10/1/2023    61245.0  234.031000  3.566667  3.878666          4.7  128.356667   
1/1/2024     57750.0  233.302000  3.633333  3.799049          4.6  129.153333   
4/1/2024     60065.0  235.674000  3.633333  3.719431          4.6  131.796667   
NaN              NaN  236.343734  3.738000  3.639814          NaN  133.660000   

           Cao-wages per ho

In [10]:
data = raw_data[['Start-ups', 
          'GDP', 
          'Job_vacancy',
          'Loans',
          'Zelfstandigenaftrek'
          ]]

In [11]:
def initialregression(data1):

    data1.dropna(inplace=True)
    data1 = np.log(data1)
    data1 = data1.diff(-4)

    data1.dropna(inplace=True)

    y = data1.pop('Start-ups')
    X = data1
    X = sm.add_constant(X)

    model = sm.OLS(y,X)
    results = model.fit()
    print(results.summary())

data1 = data.copy()
initialregression(data1)

                            OLS Regression Results                            
Dep. Variable:              Start-ups   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     2.447
Date:                Mon, 14 Oct 2024   Prob (F-statistic):             0.0565
Time:                        08:18:22   Log-Likelihood:                 43.249
No. Observations:                  62   AIC:                            -76.50
Df Residuals:                      57   BIC:                            -65.86
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0158    

In [12]:
# # one at a time
# data.dropna(inplace=True)

# data['Start_ups_percent'] = 100*((data['Start-ups']/data['Start-ups'].shift(4)) - 1)
# data['GDP_percent_lag']  = 100*((data['GDP'].shift(1)/data['GDP'].shift(5)) - 1)
# data['Job_vacancy_lag']  = data['Job_vacancy'].shift(2) - data['Job_vacancy'].shift(6)
# data['Loans']  = data['Loans'].shift(4)
# data['Zelfstandigenaftrek']  = 100*((data['Zelfstandigenaftrek']/data['Zelfstandigenaftrek'].shift(4)) - 1)

# data3 = data.dropna()
# data3 = data3.iloc[25:,]

# results = smf.ols("Start_ups_percent ~ 1 + GDP_percent_lag + Job_vacancy_lag + Loans + Zelfstandigenaftrek", data=data3).fit()

# # Inspect the results
# print(results.summary())

In [13]:
data

,Start-ups,GDP,Job_vacancy,Loans,Zelfstandigenaftrek
1/1/2006,NaN,179.989000,NaN,185.370333,6500
4/1/2006,NaN,182.917000,NaN,191.778333,6500
7/1/2006,NaN,183.853000,NaN,193.400333,6500
10/1/2006,NaN,185.449000,NaN,207.998667,6500
1/1/2007,46865.0,187.544000,NaN,230.103000,6500
...,...,...,...,...,...
7/1/2023,64570.0,233.457000,4.8,355.830667,5030
10/1/2023,61245.0,234.031000,4.7,364.335333,5030
1/1/2024,57750.0,233.302000,4.6,365.155667,3750
4/1/2024,60065.0,235.674000,4.6,370.588333,3750


In [14]:
# one at a time
data.dropna(inplace=True)

data['Start_ups_percent'] = (np.log(data['Start-ups'])).diff(4)
data['GDP_percent_lag']  = ((np.log(data['GDP'])).diff(4)).shift(1)
data['Job_vacancy_lag']  = ((np.log(data['Job_vacancy'])).diff(4)).shift(2)
data['Loans_lag']  = data['Loans'].shift(4)
data['Zelfstandigenaftrek_lag']  = (np.log(data['Zelfstandigenaftrek'])).diff(4)
data.head(20)

C:\Users\jpark\AppData\Local\Temp\ipykernel_26956\1368638342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)
C:\Users\jpark\AppData\Local\Temp\ipykernel_26956\1368638342.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Start_ups_percent'] = (np.log(data['Start-ups'])).diff(4)
C:\Users\jpark\AppData\Local\Temp\ipykernel_26956\1368638342.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,Start-ups,GDP,Job_vacancy,Loans,Zelfstandigenaftrek,Start_ups_percent,GDP_percent_lag,Job_vacancy_lag,Loans_lag,Zelfstandigenaftrek_lag
1/1/2008,33460.0,194.907,3.4,274.882667,6500,NaN,NaN,NaN,NaN,NaN
4/1/2008,47585.0,195.047,3.4,290.752000,6500,NaN,NaN,NaN,NaN,NaN
7/1/2008,40790.0,194.523,3.2,300.331333,6500,NaN,NaN,NaN,NaN,NaN
10/1/2008,43670.0,192.383,2.5,298.392667,6500,NaN,NaN,NaN,NaN,NaN
1/1/2009,28810.0,187.988,1.9,296.109000,7280,-0.149628,NaN,NaN,274.882667,0.113329
4/1/2009,56190.0,186.244,1.5,299.678667,7280,0.166221,-0.036144,NaN,290.752000,0.113329
7/1/2009,47575.0,186.439,1.5,301.614333,7280,0.153870,-0.046183,-0.581922,300.331333,0.113329
10/1/2009,55880.0,187.662,1.5,305.169667,7280,0.246545,-0.042446,-0.818310,298.392667,0.113329
1/1/2010,24560.0,187.668,1.5,309.041000,7280,-0.159603,-0.024846,-0.757686,296.109000,0.000000
4/1/2010,57665.0,189.392,1.6,313.061333,7280,0.025912,-0.001704,-0.510826,299.678667,0.000000


In [15]:

data3 = data.dropna()

data3 = data3.iloc[18:,]

results = smf.ols("Start_ups_percent ~ 1 + GDP_percent_lag + Job_vacancy_lag + Loans_lag + Zelfstandigenaftrek_lag", data=data3).fit()

# Inspect the results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      Start_ups_percent   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.442
Method:                 Least Squares   F-statistic:                     9.110
Date:                Mon, 14 Oct 2024   Prob (F-statistic):           3.16e-05
Time:                        08:18:22   Log-Likelihood:                 63.485
No. Observations:                  42   AIC:                            -117.0
Df Residuals:                      37   BIC:                            -108.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 